<a href="https://colab.research.google.com/github/Kaniz-Subarna/CodeAlpha_Language-Translation-Tool/blob/main/Language_Translation_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googletrans==4.0.0-rc1 gtts ipywidgets

In [ ]:
from googletrans import Translator, LANGUAGES
import ipywidgets as widgets
from IPython.display import display, HTML, Audio
from gtts import gTTS
import tempfile

translator = Translator()

code_to_name = {code: name.title() for code, name in LANGUAGES.items()}
code_to_name = {'auto': 'Auto-detect', **code_to_name}
options = sorted([(name, code) for code, name in code_to_name.items()], key=lambda x: x[0].lower())
name_to_code = {name: code for name, code in options}
print("Ready — language list prepared. Example languages:", options[:8])

In [ ]:
input_text = widgets.Textarea(
    value='',
    placeholder='Type text here to translate...',
    description='Input:',
    layout=widgets.Layout(width='95%', height='140px')
)

src_dropdown = widgets.Dropdown(
    options=options,
    value='auto',
    description='From:'
)

tgt_dropdown = widgets.Dropdown(
    options=[item for item in options if item[1] != 'auto'],
    value='en',
    description='To:'
)

translate_button = widgets.Button(description='Translate', button_style='primary', icon='language')
copy_button = widgets.Button(description='Copy', icon='copy')
tts_button = widgets.Button(description='Play TTS', icon='volume-up', button_style='success')
clear_button = widgets.Button(description='Clear', icon='trash')

output_area = widgets.Textarea(
    value='',
    placeholder='Translation will appear here...',
    description='Output:',
    layout=widgets.Layout(width='95%', height='140px')
)

status = widgets.Label(value='')
controls = widgets.HBox([src_dropdown, tgt_dropdown, translate_button, copy_button, tts_button, clear_button])
ui = widgets.VBox([input_text, controls, output_area, status])

In [ ]:
def do_translate(_=None):
    status.value = "Translating..."
    text = input_text.value.strip()
    if not text:
        status.value = "Enter text first."
        return
    src = src_dropdown.value
    dest = tgt_dropdown.value
    try:
        res = translator.translate(text, src=src, dest=dest)
        output_area.value = res.text
        detected = getattr(res, 'src', None)
        if src == 'auto' and detected:
            detected_name = code_to_name.get(detected, detected).title()
            status.value = f"Done — detected source: {detected_name} ({detected})"
        else:
            status.value = "Done"
    except Exception as e:
        status.value = f"Error: {e}"

def do_copy(_=None):
    txt = output_area.value
    if not txt:
        status.value = "Nothing to copy."
        return
    js = f"""
    navigator.clipboard.writeText({txt!r}).then(function() {{
      console.log('Copied to clipboard.');
    }}, function(err) {{
      console.log('Could not copy text: ', err);
    }});
    """
    display(HTML(f"<script>{js}</script>"))
    status.value = "Copied to clipboard (if your browser allows it)."

def do_tts(_=None):
    txt = output_area.value.strip()
    if not txt:
        status.value = "No translated text to speak."
        return
    dest = tgt_dropdown.value
    lang_code = dest if dest in LANGUAGES else 'en'
    try:
        status.value = "Generating audio..."
        tmp = tempfile.NamedTemporaryFile(suffix='.mp3', delete=False)
        tmp_name = tmp.name
        tmp.close()
        tts = gTTS(txt, lang=lang_code)
        tts.save(tmp_name)
        display(Audio(tmp_name, autoplay=False))
        status.value = "Audio ready (use the player that appeared above)."
    except Exception as e:
        status.value = f"TTS error: {e}"

def do_clear(_=None):
    input_text.value = ''
    output_area.value = ''
    status.value = ''

translate_button.on_click(do_translate)
copy_button.on_click(do_copy)
tts_button.on_click(do_tts)
clear_button.on_click(do_clear)

display(HTML('<h2 style="text-align:center;">Language Translation Tool</h2>'))
display(ui)
status.value = "UI ready — enter text and press Translate."